In [1]:
!pip install gym==0.17.3
!pip install matplotlib
!pip install tensorflow==2.10

  Using cached tensorflow-2.10.0-cp38-cp38-win_amd64.whl.metadata (3.1 kB)
Using cached tensorflow-2.10.0-cp38-cp38-win_amd64.whl (455.9 MB)


In [2]:
import random
import gym
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam




In [3]:

NUM_ACTIONS = 3 
action_bins = np.linspace(-2, 2, NUM_ACTIONS) 
print(action_bins)

[-2.  0.  2.]


In [ ]:
#qns to ask:
#the action bins are set to -2 to 2, so the action space is continuous, but the action space in the environment is discrete, so how do i map the action bins to the discrete action space?

#the parameters like gamma, epsilon, epsilon_min, epsilon_decay, learning_rate, alpha, etc. are they all correct?

In [10]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.99           # Discount factor -----------------ASK CHER
        self.epsilon = 1.0          # Exploration rate 
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(12, input_dim=self.state_size, activation='relu'))
        model.add(Dense(12, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))  # Q-values for each discrete action
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state[np.newaxis, :], verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
    
        states = np.zeros((batch_size, self.state_size))
        targets = np.zeros((batch_size, self.action_size))
    
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state[np.newaxis, :], verbose=0)[0])
            target_f = self.model.predict(state[np.newaxis, :], verbose=0)[0]
            target_f[action] = target
    
            states[i] = state
            targets[i] = target_f
    
        
        self.model.fit(states, targets, epochs=1, verbose=0)
    
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
env = gym.make('Pendulum-v0')  # gym 0.17.3
state_size = env.observation_space.shape[0]
action_size = NUM_ACTIONS  # number of discrete actions

agent = DQNAgent(state_size, action_size)
episodes = 500
batch_size = 64

  


In [ ]:
for e in range(episodes):
    state = env.reset()
    total_reward = 0

    for time in range(200):
        action_idx = agent.act(state)
        action = [action_bins[action_idx]]  # env expects an array

        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action_idx, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            break

    # ✅ Replay only once per episode (massively faster)
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

    print(f"Episode {e+1}: Reward = {total_reward}")

Episode 1: Reward = -1327.6533042903798
Episode 2: Reward = -1009.1838351485738
Episode 3: Reward = -1351.071907843939
Episode 4: Reward = -1275.3337342267298
Episode 5: Reward = -1321.2789465804112
Episode 6: Reward = -953.5066040334468
Episode 7: Reward = -1058.124411506806
Episode 8: Reward = -1587.7885191900361
Episode 9: Reward = -1552.8001788677125
Episode 10: Reward = -1246.5251047691333
Episode 11: Reward = -984.3597635887087
Episode 12: Reward = -991.4713378550352
Episode 13: Reward = -1560.4806425510924
Episode 14: Reward = -871.4464207944507
Episode 15: Reward = -1611.214918342844
Episode 16: Reward = -1538.2613667237495
Episode 17: Reward = -1271.8589840928696
Episode 18: Reward = -918.3264043221135
Episode 19: Reward = -1178.9412822796262
Episode 20: Reward = -1454.0159434242594
Episode 21: Reward = -885.6098170028346
Episode 22: Reward = -1175.3831843741543
Episode 23: Reward = -977.0664278144861
Episode 24: Reward = -1379.8553455679464
Episode 25: Reward = -1310.58092836